In [ ]:
# лучше тоже в обший install выше, иначе требует строку кода выше
!pip install git+https://github.com/ai-forever/diffusers.git -q
!pip install accelerate -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.3 MB/s eta 0:00:00


- https://huggingface.co/docs/diffusers/en/api/pipelines/text_to_video_zero

- https://github.com/huggingface/diffusers/blob/main/docs/source/en/api/pipelines/text_to_video_zero.md

# 8 frames

In [ ]:
import imageio
import torch
from diffusers import TextToVideoZeroPipeline

model_id = "runwayml/stable-diffusion-v1-5"
pipe = TextToVideoZeroPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

prompt = "A panda is playing guitar on times square"
result = pipe(prompt=prompt).images
result = [(r * 255).astype("uint8") for r in result]
imageio.mimsave("video.mp4", result, fps=4)


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


# chunks_generate

In [ ]:

import numpy as np

model_id = "runwayml/stable-diffusion-v1-5"
pipe = TextToVideoZeroPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
seed = 0
video_length = 16  #24 ÷ 4fps = 6 seconds
chunk_size = 8
prompt = "A panda is playing guitar on times square"

# Generate the video chunk-by-chunk
result = []
chunk_ids = np.arange(0, video_length, chunk_size - 1)
generator = torch.Generator(device="cuda")
for i in range(len(chunk_ids)):
    print(f"Processing chunk {i + 1} / {len(chunk_ids)}")
    ch_start = chunk_ids[i]
    ch_end = video_length if i == len(chunk_ids) - 1 else chunk_ids[i + 1]
    # Attach the first frame for Cross Frame Attention
    frame_ids = [0] + list(range(ch_start, ch_end))
    # Fix the seed for the temporal consistency
    generator.manual_seed(seed)
    output = pipe(prompt=prompt, video_length=len(frame_ids), generator=generator, frame_ids=frame_ids)
    result.append(output.images[1:])

# Concatenate chunks and save
result = np.concatenate(result)
result = [(r * 255).astype("uint8") for r in result]
imageio.mimsave("video_16.mp4", result, fps=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Processing chunk 1 / 3


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/48 [00:00<?, ?it/s]

Processing chunk 3 / 3


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
def t2v_zero_chunk_generate(prompt:str, out_name:str, pipe:object, video_length = 16, chunk_size = 8, seed = 0):
  # Generate the video chunk-by-chunk
  result = []
  chunk_ids = np.arange(0, video_length, chunk_size - 1)
  generator = torch.Generator(device="cuda")
  for i in range(len(chunk_ids)):
      print(f"Processing chunk {i + 1} / {len(chunk_ids)}")
      ch_start = chunk_ids[i]
      ch_end = video_length if i == len(chunk_ids) - 1 else chunk_ids[i + 1]
      # Attach the first frame for Cross Frame Attention
      frame_ids = [0] + list(range(ch_start, ch_end))
      # Fix the seed for the temporal consistency
      generator.manual_seed(seed)
      output = pipe(prompt=prompt, video_length=len(frame_ids), generator=generator, frame_ids=frame_ids)
      result.append(output.images[1:])

  # Concatenate chunks and save
  result = np.concatenate(result)
  result = [(r * 255).astype("uint8") for r in result]
  imageio.mimsave(out_name, result, fps=4)

In [ ]:
captiom = "A baby with a red nose and a pink tongue is the focus of the image"
t2v_zero_chunk_generate(captiom, "test_16.mp4", pipe)

Processing chunk 1 / 3


/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:243: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4343: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


...

Processing chunk 3 / 3


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


# Test prompt

In [ ]:
import pandas as pd

MAIN_DIR = " you main dir"
MOVI_DIR = MAIN_DIR + "/MSR_VIT_movi"
path_csv = MOVI_DIR + "/MSRVTT_31_hand_selected_caption_uniform.csv"

msr_df = pd.read_csv(path_csv)
msr_df

,key,vid_key,video_id,sentence,caption_uniform
0,ret524,msr9229,video9229,guys trying out ice cream,A man holds a chocolate-covered cookie in his ...
1,ret627,msr7212,video7212,a man and a woman are walking a dog on a beach,A man and a woman stand side by side
2,ret398,msr7917,video7917,shania twain does a closeup for her video,The image showcases a blue sky with a few clou...
3,ret656,msr7361,video7361,a woman with a camel,A woman stands beside a camelA woman stands be...
4,ret53,msr8826,video8826,lady speaking on her show,A woman in a blue jacket and white shirt stand...
5,ret671,msr8676,video8676,a girl walking down a path,A young girl stands in a grassy field holding ...
6,ret90,msr9348,video9348,women walking on the stage with different styl...,A woman with long hair dressed in a black shir...
7,ret373,msr8483,video8483,someone is mixing ingredients in a bowl,The image showcases a close-up of a red round ...
8,ret347,msr9512,video9512,a woman with blonde hair and a black shirt is ...,A blonde woman with a ponytail wearing a black...
9,ret559,msr9036,video9036,young children in red fire chief hats are guid...,A group of people stand around a large red truck


In [ ]:
from pathlib import Path

EVAL_DIR = MAIN_DIR + "/Text2Video_Zero"
HEIGHT = 512
WIDTH = 512
QTY_FARMES = 16 # 2 chunks by 8

type_predict = f"predict_chunks_H{HEIGHT}_W{WIDTH}_F{QTY_FARMES}_caption_uniform"
Path(EVAL_DIR + f"/{type_predict}").mkdir(exist_ok=True)

prompts = list(msr_df["caption_uniform"])

for idx in range(len(prompts)):
  if idx: print()
  caption = prompts[idx]
  video_name = msr_df.iloc[idx]["video_id"].split("/")[-1]+".mp4"

  print(f"\n\ntest_idx_{idx} of movi {video_name}\ncaption: {caption}\n")

  out_path = EVAL_DIR + f"/{type_predict}/{video_name}"
  t2v_zero_chunk_generate(caption, out_path, pipe)



test_idx_0 of movi video9229.mp4
caption: A man holds a chocolate-covered cookie in his hand

Processing chunk 1 / 3


/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:243: FutureWarning: `_encode_prompt()` is deprecated and it will be removed in a future version. Use `encode_prompt()` instead. Also, be aware that the output format changed from a concatenated tensor to a tuple.
  deprecate("_encode_prompt()", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4343: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


...

  0%|          | 0/48 [00:00<?, ?it/s]

Processing chunk 3 / 3


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]